In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import sys
import tempfile
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf

import os

def deepnn(x):
    with tf.name_scope('reshape'):
        x_image = tf.reshape(x, [-1, 28, 28, 1])

  # First convolutional layer - maps one grayscale image to 32 feature maps.
    with tf.name_scope('conv1'):
        W_conv1 = weight_variable([5, 5, 1, 32])
        b_conv1 = bias_variable([32])
        h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
        aa = tf.summary.histogram('weight1',W_conv1)
        ab = tf.summary.histogram('bias1',b_conv1)

  # Pooling layer - downsamples by 2X.
    with tf.name_scope('pool1'):
        h_pool1 = max_pool_2x2(h_conv1)

  # Second convolutional layer -- maps 32 feature maps to 64.
    with tf.name_scope('conv2'):
        W_conv2 = weight_variable([5, 5, 32, 64])
        b_conv2 = bias_variable([64])
        h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
        ac = tf.summary.histogram('weight2',W_conv2)
        ad = tf.summary.histogram('bias2',b_conv2)

  # Second pooling layer.
    with tf.name_scope('pool2'):
        h_pool2 = max_pool_2x2(h_conv2)

  # Fully connected layer 1 -- after 2 round of downsampling, our 28x28 image
  # is down to 7x7x64 feature maps -- maps this to 1024 features.
    with tf.name_scope('fc1'):
        W_fc1 = weight_variable([7 * 7 * 64, 1024])
        b_fc1 = bias_variable([1024])

        h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

  # Dropout - controls the complexity of the model, prevents co-adaptation of
  # features.
    with tf.name_scope('dropout'):
        keep_prob = tf.placeholder(tf.float32)
        h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

  # Map the 1024 features to 10 classes, one for each digit
    with tf.name_scope('fc2'):
        W_fc2 = weight_variable([1024, 10])
        b_fc2 = bias_variable([10])

        y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
    return y_conv, keep_prob, aa, ab, ac, ad


def conv2d(x, W):
    """conv2d returns a 2d convolution layer with full stride."""
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    """max_pool_2x2 downsamples a feature map by 2X."""
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


def weight_variable(shape):
    """weight_variable generates a weight variable of a given shape."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    """bias_variable generates a bias variable of a given shape."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)



# Import data
mnist = input_data.read_data_sets("data/fashion/", one_hot=True)

# Create the model
x = tf.placeholder(tf.float32, [None, 784])

# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 10])

# Build the graph for the deep net
y_conv, keep_prob, aa, ab, ac, ad = deepnn(x)


with tf.name_scope('loss'):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_,
                                                        logits=y_conv)
    cross_entropy = tf.reduce_mean(cross_entropy)
    ae = tf.summary.scalar('loss', cross_entropy)

with tf.name_scope('adam_optimizer'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    correct_prediction = tf.cast(correct_prediction, tf.float32)
    accuracy = tf.reduce_mean(correct_prediction)
    af = tf.summary.scalar('accuracy',accuracy)

ckpt_dir = "./ckpt_dir"
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

# saver
saver = tf.train.Saver()

# with tf.Session() as sess:
#     sess.run([tf.local_variables_initializer(), tf.global_variables_initializer()])
#     ckpt = tf.train.get_checkpoint_state(ckpt_dir)
#     if ckpt and ckpt.model_checkpoint_path:
#         print(ckpt.model_checkpoint_path)
#         saver.restore(sess, ckpt.model_checkpoint_path)
        
#     for epoch in range(60):
#         for _ in range(10):
#             batch = mnist.test.next_batch(50)
#             print('test accuracy %g' % accuracy.eval(feed_dict={
#                                              x: batch[0], 
#                                               y_: batch[1], 
#                                              keep_prob: 1.0}))
            
    
#graph_location = tempfile.mkdtemp()
#print('Saving graph to: %s' % graph_location)
#train_writer = tf.summary.FileWriter(graph_location)
#train_writer.add_graph(tf.get_default_graph())
# best = 0
# with tf.Session() as sess:
#     # 初始化变量
#     sess.run(tf.global_variables_initializer())
#     # tensorboard
#     writer = tf.summary.FileWriter('graph', sess.graph)
#     merged = tf.summary.merge([aa, ab, ac, ad, ae, af])
#     # save
#     ckpt = tf.train.get_checkpoint_state(ckpt_dir)
#     if ckpt and ckpt.model_checkpoint_path:
#         print(ckpt.model_checkpoint_path)
#         saver.restore(sess, ckpt.model_checkpoint_path)
        
#     # train
#     for epoch in range(60):
#             for _ in range(1000):
#                 batch = mnist.train.next_batch(50)
#                 train_step.run(
#                     feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
#             accuracy_validation, summary = sess.run(
#                 [accuracy, merged], 
#                 feed_dict={
#                     x: mnist.validation.images,
#                     y_: mnist.validation.labels,
#                     keep_prob: 1.0})
#             saver.save(sess,ckpt_dir + "/model.ckpt")
#             print('epoch %d, validation accuracy %s' % (
#                 epoch, accuracy_validation))
#             best = (best, accuracy_validation)[
#                 best <= accuracy_validation]

#             writer.add_summary(summary, epoch)

#     # Test trained model
#     print("best: %s" % best)



class Dataset():
    def __init__(self, dtype='uint8', is_row_iamge=False):
        
        images = np.fromfile('./images/test_image.bin', dtype=np.uint8).reshape(-1, 28, 28, 1)
        print(images.shape)
        if dtype == 'uint8':
            self.images = images
        else:
            images = images.astype(np.float32) / 255.
            self.images = images
        if is_row_iamge:
            self.images = images.reshape([-1, 784])
        self.num_of_images = 6500
        self.offset = 0
        print('共6500张图片')
    def next_batch(self, batch_size=50):
        # 返回False表示以及没有样本
        # 注意：最后一个批次可能不足batch_size 所以推荐选择6500可以整除的batch_size
        if (self.offset + batch_size) <= self.num_of_images:
            self.offset += batch_size
            return self.images[self.offset-batch_size : self.offset]
        elif self.offset < self.num_of_images:
            return self.images[self.offset : ]
        else:
            False

dataset = Dataset(dtype='float32', is_row_iamge=True)
images = dataset.next_batch()
with tf.Session() as sess:
    sess.run([tf.local_variables_initializer(), tf.global_variables_initializer()])
    ckpt = tf.train.get_checkpoint_state(ckpt_dir)
    if ckpt and ckpt.model_checkpoint_path:
        print(ckpt.model_checkpoint_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
        
    res = sess.run(y_conv, feed_dict={x:dataset.images, keep_prob:1.0})
labels = np.argmax(res, 1)
print(labels)

Extracting data/fashion/train-images-idx3-ubyte.gz
Extracting data/fashion/train-labels-idx1-ubyte.gz
Extracting data/fashion/t10k-images-idx3-ubyte.gz
Extracting data/fashion/t10k-labels-idx1-ubyte.gz
(6500, 28, 28, 1)
共6500张图片
./ckpt_dir/model.ckpt
INFO:tensorflow:Restoring parameters from ./ckpt_dir/model.ckpt
[5 1 0 ..., 0 4 4]


In [2]:
with open("data/yanzhengbin.txt","w") as f:
    for i in range(np.argmax(res, 1).shape[0]):
        f.write(str(np.argmax(res, 1)[i])+'\n')

In [ ]:
model = deepnn()
saver = ..

with 
    saver.re